In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np

import string
import itertools

import time

## Exactly the same code from NB

* define house numbers and x types
* read data
* define functions to evaluate prediction

In [7]:
# House Number and X Types
houses = ['A', 'B', 'C']
xtype_dict = {'r':'raw', 'c':'change','l':'last'}

def read_data(house, xtype):
    assert house in houses
    assert xtype in xtype_dict
    
    act_df = pd.read_csv("data/house{}_act.csv".format(house))
    sensor_df = pd.read_csv("data/house{}_sensor.csv".format(house))
    X = np.load("data/X_{}_house{}.npy".format(xtype_dict[xtype], house))
    Y = np.load("data/Y_house{}.npy".format(house))
    miu = np.load("data/mu{}_{}.npy".format(house, xtype))
    prior = np.load('data/Prior_{}.npy'.format(house))
    return act_df, sensor_df, X, Y, miu, prior

# Prediction Evaluation Functions
def precision(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TI = np.sum(pred_label==y)
        if TI != 0:
            res += (float(TP)/TI)
    return float(res)/N
def recall(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TT = np.sum(Y==y)
        if TT != 0:
            res += float(TP)/TT
    return float(res)/N
def f_score(pred_label, Y):
    p = precision(pred_label, Y)
    r = recall(pred_label, Y)
    return 2*p*r/(p+r)
def accuracy(pred_label, Y):
    res = 0
    all_label = list(set(Y))
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        res += TP
    return float(res)/len(Y)

def evaluation(house,res_label, Y):
    print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
    print 'recall of house {} is {}'.format(house,recall(res_label, Y))
    print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
    print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

## Experiment with HMM

For debugging purposes. Function written as the next step.

In [3]:
# load data
act_df, sensor_df, X, Y, miu, prior = read_data("A", "r")

In [77]:
print X.shape
print Y.shape
print miu.shape

(40006, 14)
(40006,)
(17, 14)


In [45]:
order = 2
A = miu.shape[0]
S = miu.shape[1]
K = n_activity ** order

activity_single = [int(x) for x in list(set(act_df.label))] + [0]
activity_higher = [prod for prod in itertools.product(*np.tile(activity_single, (order,1)))]

index_to_single = dict(zip(range(A), activity_single)) #ind to activity_single dict
single_to_index = dict(zip(activity_single, range(A))) #activity_single to ind dict
index_to_higher = dict(zip(range(K), activity_higher)) #ind to activity_higher dict
higher_to_index = dict(zip(activity_higher, range(K))) #activity_higher to ind dict

In [79]:
X_train = X
Y_train = Y
X_test = X
Y_test = Y
N_train = len(X)
N_test = len(X)

In [91]:
# construct transition and emission matrix
transition_count = np.zeros((K, K)) + 0.00001
emission_count = np.zeros((S, K, 2)) + 0.00001

for i in range(order-1, N_train-1):
    yi = higher_to_index[tuple(Y_train[i-order+1:i+1])]
    ynext = higher_to_index[tuple(Y_train[i-order+2:i+2])]
    transition_count[yi, ynext] += 1
    
    xi = X[i]
    
    for j in range(S): 
        emission_count[j, yi, int(xi[j])] += 1

higher_last = higher_to_index[tuple(Y_train[-order:])]
for j in range(S): 
        emission_count[j, higher_last, int(X[-1][j])] += 1

In [108]:
transition = transition_count/np.sum(transition_count, axis=1).reshape(K,1)
log_transition = np.log(np.nan_to_num(transition))

emission = emission_count/np.sum(emission_count, axis=2).reshape(S,K,1)
log_emission = np.log(np.nan_to_num(emission))

In [80]:
# initial probability
initial = np.zeros(K)
initial[higher_to_index[tuple(Y_test[:order])]] = 1
log_initial = np.log(initial)

In [82]:
# initialize T1 and T2
T1 = np.zeros((K,N_test-order+1))
T2 = np.zeros((K,N_test-order+1))

## Nice and Clean function

In [ ]:
# define function to fit HMM model
def HMM(X, Y, miu, order):
    